In [ ]:
# TODO: playable with RL
    # CHECK: CNN structure + other info (check with stable baselines maybe)
    # TODO: fix multidiscrete / multimodal input compatibility

# TODO: test without prioritized experience replay first (epsilon greedy)
# TODO: prioritized experience replay implem



# TODO: save personal gameplay
    # press anything to start
    # reset if R pressed

In [2]:
from game_gym_env import CarGameEnv

env = CarGameEnv(render_mode="human")

state, _ = env.reset()
action_space = env.action_space
done = False
while not done:
    action = action_space.sample()
    state, reward, terminated, truncated, _ = env.step(action)
    env.render()
    done = terminated or truncated
    if terminated:
        print("terminated")
    if truncated:
        print("truncated")

env.close()

truncated


In [1]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
test_in = torch.zeros((1,1,30,15))
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
pool = nn.MaxPool2d(2, 2)
x = pool(F.relu(conv1(test_in))) # 16*30*15 -> 16*15*8
print(x.size())
x = pool(F.relu(conv2(x))) # 32*15*8 -> 32*8*4
print(x.size())
x = pool(F.relu(conv3(x))) # 64*8*4 -> 64*4*2
x.size()

torch.Size([1, 16, 15, 7])
torch.Size([1, 32, 7, 3])


torch.Size([1, 64, 3, 1])

In [3]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
device

device(type='cpu')

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        # need to sample differently, see prioritized experience replay Schaul et al. 2016
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
from game_gym_env import CarGameEnv
import pygame

env = CarGameEnv()
pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [15]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        self.fc_size = 64*3*1
        self.fc_image = nn.Linear(self.fc_size, 128)  # Flatten image output
        self.fc_scalar = nn.Linear(2, 32)  # 2 scalar features
        self.fc_merge = nn.Linear(128 + 32, 256)
        self.fc_out = nn.Linear(256, sum([3,2,2,2]))  # Output Q-values for each discrete action

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, image, scalar):
        # image: 1*30*15
        x = self.pool(F.relu(self.conv1(image))) # 16*30*15 -> 16*15*7
        x = self.pool(F.relu(self.conv2(x))) # 32*15*7 -> 32*7*3
        x = self.pool(F.relu(self.conv3(x))) # 64*7*3 -> 64*3*1
        x = x.view(-1, self.fc_size)
        x = F.relu(self.fc_image(x))

        s = F.relu(self.fc_scalar(scalar))
        print(x.shape, s.shape)
        merged = torch.cat((x, s), dim=1)
        merged = F.relu(self.fc_merge(merged))
        q_values = self.fc_out(merged)
        return torch.split(q_values, [3, 2, 2, 2], dim=1)

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
REPLAY_BUFF_SIZE = 10000

In [ ]:
# modified from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
# from utils import state_batch_to_tensor
from config import *
import numpy as np
def state_batch_to_tensor(state_batch):
    images = torch.tensor(np.array([s["image"] / 255.0 for s in state_batch])).to(torch.float32)
    scalars = torch.tensor(np.array([[s["speed"].item() / CAR_TOP_SPEED, 
                                      s["boostsLeft"] / CAR_INIT_SPEED_BOOST_COUNT] 
                                    for s in state_batch])).to(torch.float32)
    return images, scalars

# from rl_hyperparams import *

policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(REPLAY_BUFF_SIZE)

steps_done = 0

def select_action(state):
    # epsilon-greedy approach
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if True: #TESTING
    # if sample > eps_threshold:
        print("STILL TESTING, EPSILON GREEDY NOT YET")
        image, scalar = state_batch_to_tensor([state])
        q_values = policy_net(image, scalar)
        with torch.no_grad():
            action = [torch.argmax(q, dim=1).detach().cpu().numpy()[0] for q in q_values]  # Take highest Q-value per action
            return action
            # return policy_net(state).max(1).indices.view(1, 1)
    else:
        return env.action_space.sample()

test_state = env.observation_space.sample()
print("action:", select_action(test_state))

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_img_batch, state_scr_batch = state_batch_to_tensor(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    print("CHECK STATE ACTION VALUES")
    state_action_values = policy_net(state_img_batch, state_scr_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values # how does this work with multidiscrete
    # Compute the expected Q values
    next_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, next_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()



torch.Size([1, 128]) torch.Size([1, 32])
action: [1, 1, 1, 1]


In [ ]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50

steps_done = 0

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            # episode_durations.append(t + 1)
            # plot_durations()
            break

In [3]:
import pygame
pygame.quit()

In [73]:
import pygame
import numpy as np

In [74]:
from car_physics import Car
from checkpoint import generateCheckpoints
from utils import get_rl_state

In [75]:
import cv2

def debugMask(mask):
    resized = mask.scale((4, 4))
    width, height = resized.get_size()
    result = np.zeros((width, height))
    for x in range(width):
        for y in range(height):
            result[x,y] = mask.get_at((x,y))
    return result

In [79]:
from racing_game import Game
from config import *
game = Game()
game.reset()

In [80]:
isAccelerating = False
isBreaking = False
steerDirection = 1
tryBoosting = False

score = 0 # curr checkpoint
timer = 0

gameStarted = False # press to start

maxSpeed = 0

In [81]:
while True:
    if not gameStarted:
        if pygame.event.peek(pygame.KEYDOWN):
            gameStarted = True
    if not gameStarted:
        game.display()
        continue
    # Process player inputs.
    tryBoosting = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            raise SystemExit
        if event.type == pygame.MOUSEBUTTONDOWN:
            # Get mouse position
            mouse_pos = pygame.mouse.get_pos()
            print(f"Mouse clicked at: {mouse_pos}")
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT: 
                steerDirection += 1
            if event.key == pygame.K_RIGHT:
                steerDirection -= 1
            if event.key == pygame.K_UP:
                isAccelerating = True
            if event.key == pygame.K_DOWN:
                isBreaking = True
            if event.key == pygame.K_SPACE:
                tryBoosting = True
            if event.key == pygame.K_PLUS:
                game.car.BASE_ACCEL += 0.01
                print(game.car.BASE_ACCEL)
            if event.key == pygame.K_MINUS:
                game.car.BASE_ACCEL -= 0.01
                print(game.car.BASE_ACCEL)
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT: 
                steerDirection -= 1
            if event.key == pygame.K_RIGHT:
                steerDirection += 1
            if event.key == pygame.K_UP:
                isAccelerating = False
            if event.key == pygame.K_DOWN:
                isBreaking = False
    game.inputProcessing(steerDirection, isAccelerating, isBreaking, tryBoosting)
    maxSpeed = max(maxSpeed, game.car.vel)
    if game.timer % FPS_RATE == 0:
        print(maxSpeed)
    game.display(DEBUG=True)
    game.timeUpdate(framerate=FPS_RATE)

0.19988
1.954787720496692
1.954787720496692
1.954787720496692
1.954787720496692
3.1623813668819154
3.1623813668819154
3.1623813668819154
3.1623813668819154
3.1623813668819154
3.1623813668819154
3.1623813668819154
3.20760214625358
3.4967443735969588
3.4967443735969588
3.4967443735969588
3.4967443735969588
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
3.530878832402872
6.1155894720685
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
6.418889331531042
7.180450033090608
7.180450033090608
7.180450033090608
7.180450033090608
7.180450033090608
7.1804500330

SystemExit: 

d:\cours\ENSTA\P2\RL\rl_env\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
pygame.quit()